In [1]:
import pandas as pd
import seaborn as sns
import math
import numpy as np
from collections import Counter
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
# Import mlflow
import mlflow

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [2]:
def net_input(w_, X):
    """Calculate net input"""
    return np.dot(X, w_[1:]) + w_[0]

def activation(self, X):
    """Compute linear activation"""
    return X

def predict(self, X):
    """Return class label after unit step"""
    return np.where(self.activation(self.net_input(X)) >= 0.0, 1, -1)

In [3]:
#Import Data

df_train = pd.read_excel("train_split.xlsx",index_col=None)
df_test = pd.read_excel("test_split.xlsx",index_col=None)

#Addestramento

X_train = df_train.drop(columns=["Transported"])
X_train[['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']] = scaler.fit_transform(X_train[['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']])
y_train = df_train["Transported"].replace(0,-1)


#test

X_test = df_test.drop(columns=["Transported"])
X_test[['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']] = scaler.fit_transform(X_test[['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']])
y_test = df_test["Transported"].replace(0,-1)

In [4]:
#Parametri
random_state = 1
eta = 0.0000001
n_iter = 40000

In [8]:
#MLFLOW

mlflow.set_tracking_uri("http://35.203.161.237:5000")
mlflow.set_experiment("Adaline-1")


2023/02/06 10:59:50 INFO mlflow.tracking.fluent: Experiment with name 'Adaline-1' does not exist. Creating a new experiment.


<Experiment: artifact_location='gs://mlflowbip_bucket/1', creation_time=1675677583221, experiment_id='1', last_update_time=1675677583221, lifecycle_stage='active', name='Adaline-1', tags={}>

In [35]:
def adaline_optimization(lista_variabili):

    eta = lista_variabili[0]
    n_iter = int(lista_variabili[1])

    random_state=1
    rgen = np.random.RandomState(random_state)
    w_ = rgen.normal(loc=0.0, scale=0.01, size=1 + X_train.shape[1])
    cost_ = []
    

    mlflow.set_tracking_uri("http://35.203.161.237:5000")
    mlflow.start_run(experiment_id= "1")
    mlflow.log_param("eta", eta)
    mlflow.log_param("n_iter", n_iter)

    for i in range(n_iter):
        output = net_input(w_ ,X_train)
        errors = (y_train - output)
        w_[1:] += eta * X_train.T.dot(errors)
        w_[0] += eta * errors.sum()
        cost = (errors**2).sum() / 2.0
        cost_.append(cost)

    score = sum((np.where(net_input(w_,X_test) >= 0.0, 1, -1)==y_test)*1)/len(np.where(net_input(w_,X_test) >= 0.0, 1, -1)-y_test)

    mlflow.log_metric("score",score)
    mlflow.end_run()
    return -1*score



In [15]:
#Addestramento ed estrapolazione pesi w_
run_number = 1
mlflow.start_run(run_name = "ADA_"+str(run_number), )

rgen = np.random.RandomState(random_state)
w_ = rgen.normal(loc=0.0, scale=0.01, size=1 + X_train.shape[1])
cost_ = []

mlflow.log_param("eta", eta)
mlflow.log_param("n_iter", n_iter)

for i in range(n_iter):
    output = net_input(w_ ,X_train)
    errors = (y_train - output)
    w_[1:] += eta * X_train.T.dot(errors)
    w_[0] += eta * errors.sum()
    cost = (errors**2).sum() / 2.0
    cost_.append(cost)

score = sum((np.where(net_input(w_,X_test) >= 0.0, 1, -1)==y_test)*1)/len(np.where(net_input(w_,X_test) >= 0.0, 1, -1)-y_test)

mlflow.log_metric("score",score)

mlflow.end_run()

#Visualizzazione simulazione

#fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(8, 4))

#ax[0].plot(range(1, len(cost_) + 1), cost_, marker='o')
#ax[0].set_xlabel('Epochs')
#ax[0].set_ylabel('Sum-squared-error')


#plt.tight_layout()
#plt.show()

In [28]:
#test di ottimizzazione
adaline_optimization(eta,4000)

[0.001, 0.0001, 1e-05, 1e-06, 1e-07, 1e-08]

In [63]:
#grid search



for n_iter in range(1000,51000,1000):
    for eta in [10**-3,10**-4,10**-5,10**-6,10**-7,10**-8]:
        adaline_optimization([eta,n_iter])


